# In The Name of GOD

# Mohammad Mahdi Shafighy

# Simple artificial neural network - Master's Practice

# ***عدالت اجتماعی در مصرف انرژی: طرح پرداخت نقدی به جای سهمیه انرژی***

# گام 1: تعریف مشکل
# در این پروژه، هدف ما پیش‌بینی مقدار مصرف سوخت و انرژی سهمیه هر خانوار است ، برای اینکه بتوان یارانه معادل برای ایرن انرژی ها راتدارک دید. این پیش‌بینی می‌تواند با استفاده از داده‌های مختلف مانند تعداد اعضای خانوار، منطقه جغرافیایی، نوع سوخت مصرفی و پارامترهای مشابه انجام گیرد

# گام 2: تولید داده‌های کلان شبه تصادفی
# برای شبیه‌سازی داده‌ها می‌توانیم از داده‌های شبه تصادفی استفاده کنیم. ابتدا داده‌های شبه‌تصادفی تولید می‌کنیم که شامل ویژگی‌هایی از خانوارها باشد، مانند تعداد اعضا، منطقه جغرافیایی، نوع سوخت، و مصرف سوخت. سپس سهمیه انرژی و معادل نقدی آن را محاسبه می‌کنیم

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)
n_samples = 1000

# Frist Model
# num_members = np.random.randint(1,8, size = n_samples)
# region = np.random.choice(['Tehran', 'Esfehan','Yazd','karag','khozestan'], size=n_samples)
# fuel_type = np.random.choice(['Natural Gas', 'Electicity','Heating Oil'], size= n_samples)
# fuel_consumption = np.random.uniform(50, 500, size=n_samples)*num_members

# data = pd.DataFrame( {
#     'num_members': num_members,
#     'region': region,
#     'fuel_type': fuel_type,
#     'fuel_consumption': fuel_consumption
# })

# Second Model
# regions = ['Tehran', 'Esfehan','Yazd','karag','khozestan']
# fuel_types = ['Natural Gas', 'Electicity','Heating Oil']
# data_list = []
# for region in regions:
#   region_fuel_types = np.random.choice(fuel_types, size=n_samples)
#   region_data = pd.DataFrame({
#       'region':[region] * n_samples,
#       'fuel_type': region_fuel_types
#   })
#   data_list.append(region_data)
# data.head(10)

data = pd.DataFrame({
    'num_members': np.random.randint(1, 8, n_samples),
    'region': np.random.choice(['Tehran','Mashhad','Isfahan','Shiraz','Tabriz','Karaj','Qom','Ahvaz','Kermanshah','Urmia','Gilan','Zahedan','Arak','Yazd','Hamedan','Kerman','Bandar Abbas','Kashan','Sanandaj','Gorgan','Khorramabad','Bojnurd','Sabzevar','Zanjan','Qazvin'], n_samples),
    'fuel_type': np.random.choice(['Natural Gas'], n_samples),
    'fuel_consumption': np.random.randint(50, 500, n_samples ,)
})
# مقدار سهمیه و معادل نقدی آن
data['quota'] = data['fuel_consumption']*0.1
data['cash_equivalent'] = data['quota']*1600

# ذخیره داده ها

In [2]:
data.to_csv('simulated_energy_data.csv', index=False)

# نمایش داده ها

In [3]:
data.head(10)

,num_members,region,fuel_type,fuel_consumption,quota,cash_equivalent
0,7,Sabzevar,Natural Gas,446,44.6,71360.0
1,4,Urmia,Natural Gas,88,8.8,14080.0
2,5,Sanandaj,Natural Gas,320,32.0,51200.0
3,7,Bandar Abbas,Natural Gas,424,42.4,67840.0
4,3,Tabriz,Natural Gas,334,33.4,53440.0
5,5,Shiraz,Natural Gas,206,20.6,32960.0
6,5,Urmia,Natural Gas,124,12.4,19840.0
7,7,Zanjan,Natural Gas,421,42.1,67360.0
8,2,Bandar Abbas,Natural Gas,337,33.7,53920.0
9,3,Urmia,Natural Gas,179,17.9,28640.0


In [4]:
data.describe()

,num_members,fuel_consumption,quota,cash_equivalent
count,1000.0000,1000.000000,1000.000000,1000.000000
mean,3.9600,274.053000,27.405300,43848.480000
std,2.0036,130.176042,13.017604,20828.166771
min,1.0000,50.000000,5.000000,8000.000000
25%,2.0000,165.750000,16.575000,26520.000000
50%,4.0000,277.000000,27.700000,44320.000000
75%,6.0000,383.250000,38.325000,61320.000000
max,7.0000,499.000000,49.900000,79840.000000


# گام 3: پیش‌پردازش داده‌ها
# **طراحی مدل محاسبه سهمیه**
# برای استفاده از این داده‌ها در مدل شبکه عصبی، باید داده‌ها را پیش‌پردازش کنیم. این شامل تبدیل داده‌های دسته‌ای  به عددی و نرمال‌سازی مقادیر است

In [5]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

X = data[['num_members', 'region', 'fuel_type', 'fuel_consumption']]
y = data['cash_equivalent']
# تبدیل داده‌های دسته‌ای و نرمال‌سازی
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), ['num_members', 'fuel_consumption']),
        ('cat', OneHotEncoder(), ['region', 'fuel_type'])
    ]
)
X_processed = preprocessor.fit_transform(X)

# تقسیم داده ها به مجموعه های آموزشی و آزمایشی

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_processed,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)


# گام 3 : مدل شبکه عصبی

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# model = Sequential([
#     Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
#     Dense(32, activation='relu'),
#     Dense(1, activation='linear')
# ])

# model.compile(optimizer='adam',
#               loss='mean_squared_error',
#               metrics=['mae'])
# history = model.fit(X_train, y_train,
#                     epochs=50, batch_size=32,
#                     validation_split=0.2)
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),  #برای منظم سازی dropout اضافه کردن
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])
# توقف زود هنگام برای جلوگیری از بیش برازش
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train,
                    epochs=200,  # افزایش
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping])  # اضافه کردن توقف زودهنگام



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 2340532480.0000 - mae: 43804.5195 - val_loss: 2301582592.0000 - val_mae: 42748.8008
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2368778496.0000 - mae: 44256.1445 - val_loss: 2300903168.0000 - val_mae: 42741.1836
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2339966720.0000 - mae: 43807.2305 - val_loss: 2298476288.0000 - val_mae: 42714.0195
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2431756544.0000 - mae: 44523.1172 - val_loss: 2291255808.0000 - val_mae: 42633.1641
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2387588608.0000 - mae: 44597.5273 - val_loss: 2272847616.0000 - val_mae: 42426.3633
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2277964544.0000 - mae: 42943.8672 - val_loss: 2233916672.0000 - val_mae: 41985.3203
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2137902720.0000 - mae: 41640.7773 - val_loss: 2161881600.0000 - val_mae: 41156.613

# ارزیابی مدل روی داده های آزمایشی

In [21]:
from sklearn.metrics import mean_squared_error, r2_score

# test_loss, test_mae = model.evaluate(X_test, y_test)
# print(f"Test loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse:.4f}")

r2 = r2_score(y_test, y_pred)
print(f"Test R-squared: {r2:.4f}")

#نتایج اولین مرحله ارزیابی مدل :
# Test MSE: 922347802.6517
# Test R-squared: -1.1994

#نتایج دومین مرحله ارزیابی مدل :
# Test MSE: 116508.7380
# Test R-squared: 0.9997

#نتایج سومین مرحله ارزیابی مدل :



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Test MSE: 116508.7380
Test R-squared: 0.9997


In [23]:
predictions = model.predict(X_test)

# نمایش نمونه‌ای از پیش‌بینی‌ها
for i in range(5):
    print(f"Real: {y_test.iloc[i]:.2f}, Predicted: {predictions[i][0]:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Real: 56160.00, Predicted: 55862.23
Real: 71200.00, Predicted: 71430.78
Real: 30880.00, Predicted: 30636.85
Real: 34560.00, Predicted: 34235.62
Real: 64160.00, Predicted: 64163.17
